In [312]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [313]:
ds_train = pd.read_csv("datasets/DailyDelhiClimateTrain.csv",
                       names=['date', 'meantemp', 'humidity', 'wind_speed', 'meanpressure', 'comfortable'], skiprows=1)
ds_test = pd.read_csv("datasets/DailyDelhiClimateTest.csv",
                      names=['date', 'meantemp', 'humidity', 'wind_speed', 'meanpressure', 'comfortable'], skiprows=1)
ds_train

,date,meantemp,humidity,wind_speed,meanpressure,comfortable
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667,0
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000,0
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667,0
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667,0
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000,0
...,...,...,...,...,...,...
1457,2016-12-28,17.217391,68.043478,3.547826,1015.565217,0
1458,2016-12-29,15.238095,87.857143,6.000000,1016.904762,0
1459,2016-12-30,14.095238,89.666667,6.266667,1017.904762,0
1460,2016-12-31,15.052632,87.000000,7.325000,1016.100000,0


In [314]:
def normalise(data, min, max):
    data = data - min
    data = data / max
    return data


ds_train.drop("date", axis=1, inplace=True)
data_train = ds_train.values
ds_train_normed = normalise(data_train, data_train.min(axis=0), data_train.max(axis=0))

ds_test.drop("date", axis=1, inplace=True)
data_test = ds_train.values
ds_test_normed = normalise(data_test, data_test.min(axis=0), data_test.max(axis=0))

ds_train_normed

array([[0.10332103, 0.71071429, 0.        , 0.13265583, 0.        ],
       [0.03616236, 0.78571429, 0.07058266, 0.13293363, 0.        ],
       [0.0301353 , 0.73571429, 0.10974262, 0.13304649, 0.        ],
       ...,
       [0.20910209, 0.76238095, 0.14842886, 0.13294727, 0.        ],
       [0.23383181, 0.73571429, 0.17349597, 0.13271226, 0.        ],
       [0.10332103, 0.86571429, 0.        , 0.13269924, 0.        ]])

In [315]:
def windowed_dataset(series, batch_size, n_past=10, n_future=10, shift=1):
    '''
    :param series: an iterable
    :param batch_size: number of samples per batch
    :param n_past: number of timesteps to look back (x or input to the model)
    :param n_future: number of timesteps to predict (y or output of the model)
    :param shift: number of timesteps to shift the window by, usually 1 if we want to maximise the use of data
    :return:
    '''
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda x: (x[:n_past, :], x[:n_past, :1]))

    # ds = ds.map(lambda w: (w[:n_past, :4], (w[:n_past, :4], w[:n_past, 4])))
    return ds.batch(batch_size).prefetch(1)

In [316]:
BATCH_SIZE = 32  # explore how changing this affects the model (it is not as simple as larger batch size = better performance, lik ein image tasks)
N_PAST = 10
N_FUTURE = 10
SHIFT = 1

EPOCHS = 100

In [317]:
proc_ds_train = windowed_dataset(series=ds_train_normed, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
proc_ds_test = windowed_dataset(series=ds_train_normed, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE,
                                shift=SHIFT)
proc_ds_train.element_spec

(TensorSpec(shape=(None, None, 5), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))

In [318]:
xIn = tf.keras.layers.Input((N_PAST, 5))
x = tf.keras.layers.LSTM(32, return_sequences=True)(xIn)
x = tf.keras.layers.LSTM(32, return_sequences=True)(x)
x = tf.keras.layers.Dense(64, activation='swish')(x)
# x = tf.keras.layers.Flatten()(x)

x_meantemp = tf.keras.layers.Dense(N_FUTURE, activation='swish', name="meantemp")(x)
x_humidity = tf.keras.layers.Dense(N_FUTURE, activation='swish', name="humidity")(x)
x_wind_speed = tf.keras.layers.Dense(N_FUTURE, activation='swish', name="wind_speed")(x)
x_meanpressure = tf.keras.layers.Dense(N_FUTURE, activation='swish', name="meanpressure")(x)

x_weather_out = tf.keras.layers.Dense(N_FUTURE, activation='swish')(
    tf.keras.layers.Concatenate()([x_meantemp, x_humidity, x_wind_speed, x_meanpressure]))
x = tf.keras.layers.Flatten()(x_weather_out)
x_comfortable_out = tf.keras.layers.Dense(N_FUTURE, activation='swish', name="comfortable")(x)

xOut = [x_weather_out, x_comfortable_out]

model = tf.keras.Model(inputs=xIn, outputs=xOut)
model.compile(loss='mse', optimizer='adam', metrics='mse')
model.summary()

Model: "model_24"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_25 (InputLayer)          [(None, 10, 5)]      0           []                               
                                                                                                  
 lstm_48 (LSTM)                 (None, 10, 32)       4864        ['input_25[0][0]']               
                                                                                                  
 lstm_49 (LSTM)                 (None, 10, 32)       8320        ['lstm_48[0][0]']                
                                                                                                  
 dense_48 (Dense)               (None, 10, 64)       2112        ['lstm_49[0][0]']                
                                                                                           

In [319]:
model.fit(proc_ds_train, validation_data=proc_ds_test, batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/100
46/46 [==============================] - 7s 41ms/step - loss: 0.3428 - dense_49_loss: 0.1879 - comfortable_loss: 0.1549 - dense_49_mse: 0.1879 - comfortable_mse: 0.1549 - val_loss: 0.1488 - val_dense_49_loss: 0.1146 - val_comfortable_loss: 0.0342 - val_dense_49_mse: 0.1146 - val_comfortable_mse: 0.0342
Epoch 2/100
46/46 [==============================] - 1s 16ms/step - loss: 0.1013 - dense_49_loss: 0.0845 - comfortable_loss: 0.0168 - dense_49_mse: 0.0845 - comfortable_mse: 0.0168 - val_loss: 0.0601 - val_dense_49_loss: 0.0542 - val_comfortable_loss: 0.0059 - val_dense_49_mse: 0.0542 - val_comfortable_mse: 0.0059
Epoch 3/100
46/46 [==============================] - 1s 16ms/step - loss: 0.0517 - dense_49_loss: 0.0415 - comfortable_loss: 0.0103 - dense_49_mse: 0.0415 - comfortable_mse: 0.0103 - val_loss: 0.0359 - val_dense_49_loss: 0.0304 - val_comfortable_loss: 0.0055 - val_dense_49_mse: 0.0304 - val_comfortable_mse: 0.0055
Epoch 4/100
46/46 [==============================] -

In [320]:
preds = model.predict(proc_ds_train)
preds[0]

array([[[ 0.14423195,  0.14862101,  0.15005642, ...,  0.14219286,
          0.1230441 ,  0.13913092],
        [ 0.03811274,  0.04873138,  0.05399081, ...,  0.04322451,
          0.02349218,  0.02993505],
        [ 0.01910932,  0.03163071,  0.03587838, ...,  0.0287363 ,
          0.01030125,  0.01106066],
        ...,
        [ 0.06930181,  0.0727781 ,  0.0775026 , ...,  0.08255975,
          0.06905741,  0.06362598],
        [ 0.19335532,  0.1870353 ,  0.18799739, ...,  0.20389186,
          0.19270526,  0.1910504 ],
        [ 0.14154667,  0.1354674 ,  0.14302225, ...,  0.14897177,
          0.13611922,  0.13751039]],

       [[ 0.08476141,  0.09205887,  0.09661368, ...,  0.08485691,
          0.06447164,  0.07748411],
        [ 0.02162302,  0.03351732,  0.03907268, ...,  0.02846817,
          0.00882754,  0.01308212],
        [ 0.05739913,  0.06828386,  0.06988461, ...,  0.0668349 ,
          0.0489446 ,  0.05090268],
        ...,
        [ 0.19434848,  0.18976821,  0.18898936, ...,  

In [321]:
def unnormalise(data, min, max):  # https://en.wikipedia.org/wiki/Feature_scaling
    data = data * max
    data = data + min
    return data

preds_unnormed = unnormalise(preds, data_train.min(axis=0), data_train.max(axis=0))
preds_unnormed[0]

C:\Users\lzh7522\AppData\Local\Temp\ipykernel_7732\2196605390.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = data * max


ValueError: could not broadcast input array from shape (1443,10,10) into shape (1443,10)